## CoNVO

### Context  
  
Airbnb, as “Air Bed and Breakfast”, provides an online marketplace to allow property owners to rent out their spaces for people to stay. “Inside Airbnb” collects information on properties rented and data of renting activities from Airbnb. Its goals are to give insights on Airbnb business and improve property owners and renters' experiences during their uses of Airbnb. The Chief Executive Officer makes the business decision and report to the stakeholders.  
  
### Need  
  
Airbnb needs to improve the method to determine the listing price of a property for rent. Factors influencing the listing price need to be identified. Using the data collected by “Inside Airbnb” and other data sources, and building up a better model using these factors to predict the listing price of properties for rent, the company will generate more reasonable listing prices and this will attract more users including both property owners and renters.  
  
### Vision  
  
The Chief Executive Officer will get a set of spreadsheets that contain the data categorized by property information and renting activities based on transaction history. Models using different factors to predict listing prices of properties will be generated. The best model will be selected based on the evaluation.  
  
### Outcome  
  
Factors in the best model that predict listing prices will be focused in the future. Different aspects of these factors will be investigated and conresponding data will be collected for further exploration. A report of a discussion of factors impacting listing prices will be sent to the Chief Executive Officer and will be discussed at the next Business meeting. The results will be reported to stakeholders.  
  
## Question/Problem to Answer  
  
What factors determine listing price of properties for rent on Airbnb?  
  
## Target Variable  
  
price
  
## Variables  
  
Variables available are generally information on the price, locaton, neighborhood, host's responses, reviews and property types.
  
Variables we picked are:  
  
* price: Listing price of a property for rent
* bedrooms: Number of bedrooms in a property
* beds: Number of beds in a property
* accommodates: Maximum capacity of a property
* host_is_superhost: Whether the host is a superhost which means 4.8 or higher average overall rating based on reviews
* host_response_time: Time range of host's response
* host_response_rate: Percentage of host's response
* host_acceptance_rate: Percentage of host's acceptance
* room_type: Type of room of a property
* property_type: Type of property
* number_of_reviews: Number of reviews of a property
* review_scores_rating: Scores of rated property based on reviews
* reviews_per_month: Average number of monthly reviews of a property
* num_baths: Number of bathromms in a property
* instant_bookable: Whether instant booking is available
* neighborhood_label: Label of a neighborhood based on tax-assessed values
* bath_type: Type of bathrooms  
  
## CLD  
   
A separate image file of our CLD is included in our submission. Here are the summary of relationships between price and individual variables:

* bedrooms: as we saw in the EDA, there is a low positive relationship and that Spearman's suggests that the relationship may not be linear.
* beds: as we saw in the EDA, there is a low positive relationship and that Spearman's suggests that the relationship may not be linear.
* accommodates: as we saw in the EDA, there is a moderate positive relationship and that Spearman's suggests that the relationship may be linear.
* host_is_superhost: as we saw in the EDA, there is a very weak relationship and that Spearman's suggests that the relationship may not be linear.
* host_response_time: as we saw in the EDA, there is a very weak to low relationship and that Spearman's suggests that the relationship may not be linear.
* host_response_rate: as we saw in the EDA, there is a very weak positive relationship and that Spearman's suggests that the relationship may not be linear.
* host_acceptance_rate: as we saw in the EDA, there is a very weak positive relationship and that Spearman's suggests that the relationship may not be linear.
* room_type: as we saw in the EDA, there is a very weak to weak relationship and that Spearman's suggests that the relationship may or may not be linear.
* property_type: as we saw in the EDA, there is a very weak relationship and that Spearman's suggests that the relationship may not be linear.
* number_of_reviews: as we saw in the EDA, there is a very weak negative relationship and that Spearman's suggests that the relationship may not be linear.
* review_scores_rating: as we saw in the EDA, there is a very weak negative relationship and that Spearman's suggests that the relationship may not be linear.
* reviews_per_month: as we saw in the EDA, there is a very weak positive relationship and that Spearman's suggests that the relationship may not be linear.
* num_baths: as we saw in the EDA, there is a low positive relationship and that Spearman's suggests that the relationship may not be linear.
* instant_bookable: as we saw in the EDA, there is a very weak relationship and that Spearman's suggests that the relationship may not be linear.
* neighborhood_label: as we saw in the EDA, there is a very weak relationship and that Spearman's suggests that the relationship may not be linear.
* bath_type: as we saw in the EDA, there is a very weak relationship and that Spearman's suggests that the relationship may not be linear.  
  
## Interesting Variables  
  
We explored single variable distributions and pair-wise variables for their correlations with the target variable price in our EDA. The most interesting variables are those that have at least a low correlation (Pearson's Correlation Coefficient r >= 0.30) with the target variable price. They are:  
  
* bedrooms: The bedrooms variable is a numerical integer value. It represents the number of bedrooms of properties. More bedrooms generally corresponds to a higher capacity for overnight guests. 75% of AirBnB listings in the DC area contain two or fewer bedrooms. There is one AirBnB that has a maximum number of bedrooms of 15. The distribution of the number of bedrooms appears to be right-skewed. This makes sense, as one or two bedrooms may be common for most homes, with 3 or more bedrooms being less commons in homes, especially in a densely-populated area. There is a low positive relationship (r = 0.39) between bedrooms and price and that Spearman's suggests that the relationship may not be linear.  
  
* beds: The variable beds is a numerical integer value. It represents the number of beds in the AirBnb listed property. It may correlate with bedrooms since number of bedrooms usually equals to that of beds except for rare cases. Thus, we can say about 75% of AirBnB listings in the DC area contain two or fewer beds. There is one AirBnB that has a maximum number of beds of 15.  More beds generally corresponds to a higher capacity for overnight guests. There is a low positive relationship (r = 0.42) between beds and price and that Spearman's suggests that the relationship may not be linear.  
  
* accommodates: The accommodates variable is a numerical integer value. It represents the number of people who can stay in the AirBnb listed property. More accommodates generally corresponds to a higher capacity for overnight guests. 75% of AirBnB listings in the DC area are capable of accomodating 4 people. This unsurprisingly folows a similar pattern to both the beds and bedroom variables. The data are right-skewed. Even number accommodates values are more common, which makes sense given that one bed is often counted as accommodating 2 guests. There is a moderate positive relationship (r = 0.51) between accommodates and price and that Spearman's suggests that the relationship may be linear.  
  
* num_baths: The number of bathrooms in an AirBnB is a categorical value, since AirBnbs can have "shared" or "private" bathrooms. Additionally, there may be "half-baths" as well. The most common bathroom type for Airbnb is "1 bath". There is a low positive relationship (r = 0.47) between num_baths and price and that Spearman's suggests that the relationship may not be linear.  
  
* The best ones that worked out are: 

## Null Model / Baseline Model  
  
As shown in our EDA, there are 8626 listings between Q2 2022- Q1 2023 in Washington D.C. The average price of these AirBnBs is 166 US dollars with a standard deviation of 130 US dollars. The mininum price for a place is 0 dollars and the max is 999 dollars. We can definitely see a skew right since the median (130) < mean (166).  
  
As noted in our EDA, the distribution of this variable is skewed right and not normally distributed. It is important to understand the context of why the data is being generated like this. The data does not contains a "bell shaped curve" that is partly tied to the central limit theorem. That is, the average of the observations of this variable with finite moment and 2nd moment do not converges to the normal distribution as the number of sample increases. The data is postively skewed. That is the median of the observations is lower than the mean. There are more observations on the lower 50% percentile of the dataset than the upper 50% percentile. In this situaiton, there is no symmetry in the bell curve which violates the normality assumptions and takes us to transform the distribution we want to fit for our model. Since the data reflects the time of a pricing for a particular listing. This might be considered as the spatial time between events and it makes sense.  
  
### Data Generating Story  
  
The price listing of Airbnbs varies significantly based on the demand for that specific unit. With this said, it can be noted that the demand for Airbnbs is high for Airbnbs that are cheaper and there is gradually less demand for Airbnbs that are more expensive. However, it is important to note that there is a point in the Airbnbs where the demand is as low as expensive Airbnb. That is, Airbnbs that are too cheap, people do not want to rent this Airbnb. Thus, as mentioned, because these prices are within a timeframe and are independent from each other, this distribution is most likely an exponential distribution or a family of the exponential distribution.  
  
### Null Model / Baseline Model  
  
We would predict that the mean listing price is 166.80 dollars. The standard deviation of the price using 75% error bounds is (-94.82, 428.43).  
  
## Final Model  
  
Our final model to predict listing price of a property based on differen factors is:  
  
Model: price ~ bedrooms_log + beds_log + accommodates_log + host_response_rate_log + host_acceptance_rate_log + number_of_reviews_log + review_scores_rating_log + num_baths_log + superhost_t + host_response_time_within_a_day + host_response_time_within_a_few_hours + room_type_Entire_home_apt + room_type_Private_room + property_type_apt + property_type_hotel + property_type_house + instant_bookable_t + neighborhood_high + bath_type_private + bedrooms:num_baths + bedrooms:accommodates 
  
The final linear regression model of price contains 19 variables (bedrooms_log, beds_log, accommodates_log, host_response_rate_log, host_acceptance_rate_log, number_of_reviews_log, review_scores_rating_log, num_baths_log, superhost_t, host_response_time_within_a_day, host_response_time_within_a_few_hours, room_type_Entire_home_apt, room_type_Private_room, property_type_apt, property_type_hotel, property_type_house, instant_bookable_t, neighborhood_high, and bath_type_private) and 2 interaction terms (bedrooms:num_baths and bedrooms:accommodates).  
  
The $R^2$ is 39% which is ok. All the variables in the model make sense. The error of the regression ($\sigma$) is 102.23 compared to 130.82 in th null model.
  
What the model means:

* bedrooms_log - for every unit of log bedrooms, add 11.23 from price.
* beds_log - for every unit of log beds, add 5.46 from price. 
* accommodates_log - for every unit of log accomodates, add 27.16 from price.
* host_response_rate_log - for every unit of log host_response_rate, subtract 2.64 from price. 
* host_acceptance_rate_log - for every unit of log host_acceptance_rate, add 4.5 from price.
* number_of_reviews_log - for every unit of log number_of_reviews, subtract 4.85 from price.
* review_scores_rating_log - for every unit of log review_scores_rating, subtract 12.01 from price.
* num_baths_log - for every unit of log num_baths, add 116.32 from price.
* superhost_t - for every unit of superhost_t, add 16.44 from price.
* host_response_time_within_a_day - for every unit of host_response_time_within_a_day, subtract 12.36 from price.
* host_response_time_within_a_few_hours - for every unit of host_response_time_within_a_few_hours, subtract 31.49 from price.
* room_type_Entire_home_apt - for every unit of room_type_Entire_home_apt, add 84.23 from price.
* room_type_Private_room - for every unit of room_type_Private_room, add 52.24 from price.
* property_type_apt - for every unit of property_type_apt, add 50.14 from price.
* property_type_hotel - for every unit of property_type_hotel, add 186.49 from price.
* property_type_house - for every unit of property_type_house, add 38.04 from price.
* instant_bookable_t - for every unit of instant_bookable_t, add 8.16 from price.
* neighborhood_high - for every unit of neighborhood_high, add 18.56 from price.
* bath_type_private - for every unit of bath_type_private, add 9.57 from price.
* bedrooms:num_baths - for every unit of interacction between bedrooms and num_baths, subtract 4.73 from price.
* bedrooms:accommodates - for every unit of interacction between bedrooms and accomodates, add 2.46 from price.
 
From mathematical point of view, when x = 0, the price is -14.80. Our final residual histogram shows an approximately Gamma distribution.  
  
## Prediction Examples  
  
### Prediction 1  
  
We want a place with the following conditions:  
  
* 2 bedrooms
* 3 beds
* 4 accomodates
* 98 host_response_rate
* 88 host_aceptantce_rate
* 25 number of review
* 4.8 review_scores_rating
* 1 num_baths
* Is a superhost
* host_response_time within_a_day 
* room_type Entire_home_apt 
* instant_bookable
* neighborhood value high 
* bath_type_private  
  
Our model prediction is that the listing price of this place is 405.89. It is a reasonable prediction, as on the Airbnb listing, a similar place (except for 2 baths) was listed for 427 dollars/night (https://www.airbnb.com/rooms/31809354?adults=4&check_in=2023-05-09&check_out=2023-05-13&federated_search_id=682ad4ce-c60d-43e4-94aa-5a109ef0c022&source_impression_id=p3_1683471292_jq2er3N0LFHgI7Nw). Since we only require 1 bath, it makes sense to be a bit cheaper than this listing.
  
### Prediction 2  
  
We want a place with the following conditions:  
  
* 3 bedrooms
* 3 beds
* 3 accomodates
* 98 host_response_rate
* 88 host_aceptantce_rate
* 25 number of review
* 4.8 review_scores_rating
* 2 num_baths
* Is a superhost
* host_response_time within_a_few_hours 
* room_type Entire_home_apt 
* property_type hotel
* instant_bookable
* bath_type_private  
  
Our model prediction is that the listing price of this place is 588.62. It is a reasonable prediction, as on the Airbnb listing, a similar place (except for 2 beds) was listed for 528 dollars/night (https://www.airbnb.com/rooms/863249879217789768?adults=3&category_tag=Tag%3A8678&enable_m3_private_room=true&search_mode=flex_destinations_search&check_in=2023-05-12&check_out=2023-05-13&federated_search_id=6a7bd9b6-c78c-442d-9dce-44805e0b6a11&source_impression_id=p3_1683472038_aV5p%2F9yW6GZdpUuO). Since we only require 3 beds, it makes sense to be a bit more expensive than this listing.
  
### Prediction 3  
  
* 1 bedrooms
* 2 beds
* 1 accomodates
* 98 host_response_rate
* 88 host_aceptantce_rate
* 25 number of review
* 4.8 review_scores_rating
* 1 num_baths
* Is a superhost
* host_response_time within_a_few_hours 
* room_type Private_room 
* property_type hotel
* instant_bookable
* bath_type private  
  
Our model prediction is that the listing price of this place is 415.53. It seems a bit too high since most of the places with the similar criteria are listed with a lower prices. As our $R^2$ is 39%, this is expected.